# Group ETL project - Because we are happy

#### This project is to practice ETL procedures. We will be using data from Kaggle regarding World Happiness Rankings for 2016 and 2017
#### https://www.kaggle.com/unsdsn/world-happiness#2016.csv and here: https://www.kaggle.com/unsdsn/world-happiness#2017.csv

In [1]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
# read 2016 Happiness CSV file and put into dataframe
happiness_2016_file = "Resources/2016.csv"
happiness_2016_df = pd.read_csv(happiness_2016_file)
happiness_2016_df.head()

,Country,Region,Happiness Rank,Happiness Score,Lower Confidence Interval,Upper Confidence Interval,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Denmark,Western Europe,1,7.526,7.460,7.592,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939
1,Switzerland,Western Europe,2,7.509,7.428,7.590,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463
2,Iceland,Western Europe,3,7.501,7.333,7.669,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137
3,Norway,Western Europe,4,7.498,7.421,7.575,1.57744,1.12690,0.79579,0.59609,0.35776,0.37895,2.66465
4,Finland,Western Europe,5,7.413,7.351,7.475,1.40598,1.13464,0.81091,0.57104,0.41004,0.25492,2.82596


In [3]:
# read 2017 Happiness CSV file and put into dataframe
happiness_2017_file = "Resources/2017.csv"
happiness_2017_df = pd.read_csv(happiness_2017_file)
happiness_2017_df.head()

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


### Transform happiness_2016_df

In [4]:
# Copy the columns
happiness_2016_df_transformed = happiness_2016_df[["Country", "Happiness Rank", "Happiness Score", 
                                           "Economy (GDP per Capita)", "Health (Life Expectancy)", 
                                           "Freedom", "Generosity", "Trust (Government Corruption)"]].copy()

# Rename the column headers
happiness_2016_df_transformed = happiness_2016_df_transformed.rename(columns={"Country": "country_2016", 
                                                                              "Happiness Rank": "happiness_rank_2016",
                                                                              "Happiness Score": "happiness_score_2016", 
                                                                              "Economy (GDP per Capita)": "economy_gdp_per_capita_2016", 
                                                                              "Health (Life Expectancy)": "health_life_expectancy_2016", 
                                                                              "Freedom": "freedom_2016", 
                                                                              "Generosity": "generosity_2016", 
                                                                              "Trust (Government Corruption)": "trust_government_corruption_2016"})   



# Remove possible duplicate countries (there were no duplicate coutries)
happiness_2016_df_transformed.drop_duplicates("country_2016", inplace=True)

# Remove possible duplicates rankings (this did reduce the count, 
# but we realized that we want to know of countries with the same ranking)
#happiness_2016_df_transformed.drop_duplicates("happiness_rank_2016", inplace=True)


# Round to two decimal points
happiness_2016_df_transformed = happiness_2016_df_transformed.round({"happiness_score_2016": 2, "economy_gdp_per_capita_2016": 2, "health_life_expectancy_2016": 2,
                                                                     "health_life_expectancy_2016": 2, "freedom_2016": 2, "generosity_2016": 2,
                                                                     "trust_government_corruption_2016": 2})
                                                                                                   
# Show transformed dataframe
happiness_2016_df_transformed

,country_2016,happiness_rank_2016,happiness_score_2016,economy_gdp_per_capita_2016,health_life_expectancy_2016,freedom_2016,generosity_2016,trust_government_corruption_2016
0,Denmark,1,7.53,1.44,0.80,0.58,0.36,0.44
1,Switzerland,2,7.51,1.53,0.86,0.59,0.28,0.41
2,Iceland,3,7.50,1.43,0.87,0.57,0.48,0.15
3,Norway,4,7.50,1.58,0.80,0.60,0.38,0.36
4,Finland,5,7.41,1.41,0.81,0.57,0.25,0.41
5,Canada,6,7.40,1.44,0.83,0.57,0.45,0.31
6,Netherlands,7,7.34,1.46,0.81,0.55,0.47,0.30
7,New Zealand,8,7.33,1.36,0.83,0.58,0.49,0.42
8,Australia,9,7.31,1.44,0.85,0.57,0.47,0.32
9,Sweden,10,7.29,1.45,0.83,0.58,0.38,0.41


### Transform happiness_2017_df

In [5]:
happiness_2017_df_transformed = happiness_2017_df[["Country", "Happiness.Rank", "Happiness.Score", "Economy..GDP.per.Capita.", 
                                                   "Health..Life.Expectancy.", "Freedom", "Generosity", 
                                                   "Trust..Government.Corruption."]].copy()

# Rename the column headers
happiness_2017_df_transformed = happiness_2017_df_transformed.rename(columns={"Country": "country_2017",
                                                                              "Happiness.Rank": "happiness_rank_2017",
                                                                              "Happiness.Score": "happiness_score_2017",
                                                                              "Economy..GDP.per.Capita.": "economy_gdp_per_capita_2017",
                                                                              "Health..Life.Expectancy.": "health_life_expectancy_2017",
                                                                              "Freedom": "freedom_2017",
                                                                              "Generosity": "generosity_2017",
                                                                              "Trust..Government.Corruption.": "trust_government_corruption_2017"})



# Remove possible duplicate countries (there were no duplicate coutries)
happiness_2017_df_transformed.drop_duplicates("country_2017", inplace=True)

# Remove possible duplicates rankings (there were no duplicate rankings)
# happiness_2017_df_transformed.drop_duplicates("happiness_rank_2017", inplace=True)


# Round to two decimal points
happiness_2017_df_transformed = happiness_2017_df_transformed.round({"happiness_score_2017": 2, "economy_gdp_per_capita_2017": 2, "health_life_expectancy_2017": 2,
                                                                     "health_life_expectancy_2017": 2, "freedom_2017": 2, "generosity_2017": 2,
                                                                     "trust_government_corruption_2017": 2})

happiness_2017_df_transformed

,country_2017,happiness_rank_2017,happiness_score_2017,economy_gdp_per_capita_2017,health_life_expectancy_2017,freedom_2017,generosity_2017,trust_government_corruption_2017
0,Norway,1,7.54,1.62,0.80,0.64,0.36,0.32
1,Denmark,2,7.52,1.48,0.79,0.63,0.36,0.40
2,Iceland,3,7.50,1.48,0.83,0.63,0.48,0.15
3,Switzerland,4,7.49,1.56,0.86,0.62,0.29,0.37
4,Finland,5,7.47,1.44,0.81,0.62,0.25,0.38
5,Netherlands,6,7.38,1.50,0.81,0.59,0.47,0.28
6,Canada,7,7.32,1.48,0.83,0.61,0.44,0.29
7,New Zealand,8,7.31,1.41,0.82,0.61,0.50,0.38
8,Sweden,9,7.28,1.49,0.83,0.61,0.39,0.38
9,Australia,10,7.28,1.48,0.84,0.60,0.48,0.30


### Create database connection

In [6]:
happiness_connection_string = "postgres:postgres@localhost:5432/happiness_db"
engine = create_engine(f"postgresql://{happiness_connection_string}")

In [7]:
engine.table_names()

['happiness_2016', 'happiness_2017']

### Load DataFrames into database

In [8]:
happiness_2016_df_transformed.to_sql(name='happiness_2016', con=engine, if_exists='append', index=False)

In [9]:
happiness_2017_df_transformed.to_sql(name='happiness_2017', con=engine, if_exists='append', index=False)

### Additional learning / steps:
#### When loading the files we encountered the following issues:
##### 	    - Needed to set the index to False
##### 	    - Columns that were slightly off from what we created in PGAdmin > we renamed in PGAdmin
##### 	    - We found that if we used capitals in the dataframe name, it would not allow us to insert into the database > renamed in PGAdmin 
#####        - We found that when rounding in our dataframe was superficial when using .round(2). Needed to specify columns to round.
